In [1]:
import os
import matplotlib.pyplot as plt
import torch
from PIL import Image
from torchvision import transforms
import pickle

In [2]:
train_dir = ["split0","split1","split2","split3"]
img_locs = []
for dir in train_dir:
    for cl in os.listdir(dir):
        for img_name in os.listdir(os.path.join(dir,cl)):
            img_locs.append(os.path.join(dir,cl,img_name))

print(f"The number of training images is {len(img_locs)}")

The number of training images is 1692


In [3]:
def Patch_Generator(img_loc):
    im = Image.open(img_loc).convert('L') #PIL object in grayscale
    h,w = im.size
    
    all_patches = torch.empty((1,80,80))
    
    for scale in [1, 0.9, 0.8, 0.7]:
        img = transforms.Compose([transforms.Resize((int(scale*h),int(scale*w))),transforms.ToTensor()])(im) #of shape (1,scale*h,scale*w)

        kh, kw = 80, 80  # kernel size
        dh, dw = 80, 80  # stride

        patches = img.unfold(1, kh, dh).unfold(2, kw, dw) #of shape (1,h_patch_patch,w_patch_num,kh,kw)
        patches = patches.contiguous().view(-1, kh, kw) #of shape (num_patches,kh,kw)
        all_patches = torch.concat([all_patches,patches],dim=0)
    
    return all_patches[1:] #of shape (total_num_patches,kh,kw)


In [4]:
all_augs = torch.empty((1,80,80))

#Iterating through all the images in the directory
for im_loc in img_locs:
    img_augs = Patch_Generator(im_loc)
    all_augs = torch.concat([all_augs,img_augs],dim=0)
    print(str(img_augs.shape[0])+" generated augmentations for "+im_loc)

4 generated augmentations for split0/covid/Cov-Atlas-Day+2.gif_frame69.jpg
7 generated augmentations for split0/covid/Cov-Atlas-+(43).gif_frame21.jpg
7 generated augmentations for split0/covid/Cov-Atlas-+(43).gif_frame33.jpg
4 generated augmentations for split0/covid/Cov-Atlas-Day+2.gif_frame39.jpg
4 generated augmentations for split0/covid/Cov-Atlas-Day+2.gif_frame15.jpg
21 generated augmentations for split0/covid/Cov-Atlas-Day+3.gif_frame3.jpg
21 generated augmentations for split0/covid/Cov-Atlas-Day+3.gif_frame21.jpg
7 generated augmentations for split0/covid/Cov-Atlas-+(43).gif_frame3.jpg
21 generated augmentations for split0/covid/Cov-Atlas-Day+3.gif_frame0.jpg
21 generated augmentations for split0/covid/Cov-Atlas-Day+3.gif_frame6.jpg
4 generated augmentations for split0/covid/Cov-Atlas+(44).gif_frame24.jpg
66 generated augmentations for split0/covid/Cov-Atlas-Day+1.gif_frame12.jpg


351 generated augmentations for split0/covid/Cov_Oliviera_2020_Fig15A.jpg
21 generated augmentations for split0/covid/Cov-Atlas-Day+3.gif_frame12.jpg
21 generated augmentations for split0/covid/Cov-Atlas-Day+3.gif_frame9.jpg
4 generated augmentations for split0/covid/Cov-Atlas+(44).gif_frame27.jpg
7 generated augmentations for split0/covid/Cov-Atlas+(45).gif_frame6.jpg
4 generated augmentations for split0/covid/Cov-Atlas-Day+2.gif_frame3.jpg
7 generated augmentations for split0/covid/Cov-Atlas-+(43).gif_frame36.jpg
21 generated augmentations for split0/covid/Cov-Atlas-Day+3.gif_frame18.jpg
4 generated augmentations for split0/covid/Cov-Atlas-Day+2.gif_frame45.jpg
4 generated augmentations for split0/covid/Cov-Atlas-Day+2.gif_frame42.jpg
7 generated augmentations for split0/covid/Cov-Atlas-+(43).gif_frame6.jpg
4 generated augmentations for split0/covid/Cov-Atlas+(44).gif_frame21.jpg
4 generated augmentations for split0/covid/Cov-Atlas-Day+2.gif_frame24.jpg
4 generated augmentations for 

In [5]:
print("The number of augmentations generated is "+str(all_augs.shape[0]-1))

The number of augmentations generated is 129664


In [6]:
dbfile = open('Covid_train.pkl', 'wb')
# source, destination
pickle.dump(all_augs[1:], dbfile)                    
dbfile.close()


In [11]:
a = torch.randn((1,160,160))
patches = a.unfold(1, 80, 40).unfold(2, 80, 40) #of shape (1,h_patch_patch,w_patch_num,kh,kw)
patches = patches.contiguous().view(-1, 80, 80) #of shape (num_patches,kh,kw)
patches.shape

torch.Size([9, 80, 80])